## 一、模块功能概述
本模块包含两个主函数：
- `allocate_booths`：基于贪心策略的基本展位分配算法；
- `allocate_booths_optimized`：优先满足最小需求的优化分配算法。

输入包括展位日程（`booth_schedule`）与企业需求（`enterprise_df`），输出为每日各展位的分配安排。


## 二、模型符号说明（与论文统一）

| 符号                                                    | 含义                                 |
|-------------------------------------------------------|------------------------------------|
| $ \mathcal{E} = \{e_1, e_2, \dots, e_7\} $            | 参展企业集合                             |
| $ d_i $                                               | 企业 $ e_i $ 的展位总需求时长                |
| $ g_i $                                               | 企业 $ e_i $ 可容忍的最大时间缺额              |
| $ \mathcal{B}_t = \{b_{t1}, b_{t2}, \dots, b_{tn}\} $ | 第 $ t $ 天的展位集合                     |
| $ h_b $                                               | 展位 $ b $ 的每日可用时长                   |
| $ A_{it} \subseteq \mathcal{B}_t $                    | 第 $ t $ 天分配给企业 $ i $ 的展位集合         |
| $ y_{ijt} \in \{0,1\} $                               | 第 $ j $ 个展位在第 $ t $ 天是否分配给企业 $ i $ |
| $$ a_i $$                                             | 企业$$ e_i $$当前已累计获得的总展位时长（已分配）      |
| $$ r_i = d_i - a_i $$                                 | 企业$$ e_i$$ 的剩余需求时长                 |
| $$ p_i $$                                             | 企业 $$ e_i $$ 在加权分配中的得分比例           |
| $$ R $$                                               | 所有剩余需求 $$ r_i > 0 $$ 的企业集合         |

约束条件：
- 每个展位每天只能分配给一个企业：$ \sum_i y_{ijt} \le 1 $
- 每个企业分配展位总时长不超过其需求：$ \sum_{t,j} y_{ijt} h_{b_{tj}} \le d_i $
- 每个企业至少满足：$ \sum_{t,j} y_{ijt} h_{b_{tj}} \ge d_i - g_i $

## 三、基础分配函数 `allocate_booths`

### 数学模型描述：

每次从企业集合中选取：
$$
i^* = \arg\max_i \left\{ d_i - \sum_{t,j} y_{ijt} h_{b_{tj}} \right\}
$$
优先将可用展位分配给需求剩余较大的企业
若展位时长 $$ r_i \ge 0.5 \cdot h_b $$，则允许分配。

### 实现逻辑：
优先为剩余需求量最大的企业分配展位。

In [ ]:
from collections import defaultdict
import random

def allocate_booths(booth_schedule, enterprise_df):
    # 初始化剩余需求
    demand_map = {
        row['企业代号']: {
            'remaining': row['需求小时数'],
            'min_required': row['需求小时数'] - row['可允许不足小时数']
        }
        for _, row in enterprise_df.iterrows()
    }

    assignment = {}
    for day, areas in booth_schedule.items():
        assignment[day] = {}
        for area, booths in areas.items():
            assignment[day][area] = []
            for booth in booths:
                # 选择剩余需求最多的企业
                candidates = sorted(demand_map.items(), key=lambda x: -x[1]['remaining'])
                for ent, info in candidates:
                    ## 若企业剩余需求至少能用上该展位一半以上，则允许分配
                    if info['remaining'] >= booth['时长'] * 0.5:  # 至少满足50%才分配
                        assignment[day][area].append({
                            '企业代号': ent,
                            '时长': booth['时长']
                        })
                        demand_map[ent]['remaining'] -= booth['时长']
                        break
    return assignment

## 四、优化分配函数 `allocate_booths_optimized`

### 数学模型描述：
优先满足企业最小需求：
$$
i^* = \arg\min_i \left( d_i - g_i - \sum_{t,j} y_{ijt} h_{b_{tj}} \right)
$$
若无完全匹配，则允许部分分配：
$$
h_b' = \min(h_b, d_i - a_i)
$$

### 实现逻辑：
优先保障企业的最小展出需求时间 $$ d_i - g_i $$。

In [ ]:

def allocate_booths_optimized(booth_schedule, enterprise_df):
    """
    优化版展位分配，优先满足剩余需求紧张的企业，
    确保分配时间 >= 需求-可容忍，且不超过总展位限制。
    """
    demand = {
        row['企业代号']: {
            'remaining': row['需求小时数'],
            'min_required': row['需求小时数'] - row['可允许不足小时数']
        }
        for _, row in enterprise_df.iterrows()
    }

    assignment = {}
    for day, areas in booth_schedule.items():
        assignment[day] = {}
        for area, booths in areas.items():
            assignment[day][area] = []
            for booth in booths:
                # 按照 (剩余时间 - 最小需求) 排序，优先剩余时间最少的企业
                candidates = sorted(
                    demand.items(),
                    key=lambda x: (x[1]['remaining'] - x[1]['min_required'])
                )
                assigned = False
                for ent, info in candidates:
                    if info['remaining'] >= booth['时长']:
                        assignment[day][area].append({'企业代号': ent, '时长': booth['时长']})
                        demand[ent]['remaining'] -= booth['时长']
                        assigned = True
                        break
                if not assigned:
                    # 若无企业满足完全需求，考虑分配给剩余时间>0的企业（部分满足）
                    for ent, info in candidates:
                        if info['remaining'] > 0:
                            length = min(booth['时长'], info['remaining'])
                            assignment[day][area].append({'企业代号': ent, '时长': length})
                            demand[ent]['remaining'] -= length
                            assigned = True
                            break
                if not assigned:
                    # 无可分配企业，展位空置（或者根据需求自由调整）
                    pass
    return assignment